In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Hutchinson-Gilford_Progeria_Syndrome/GSE84351'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Reprogramming Hutchinson-Gilford Progeria Syndrome fibroblasts resets epigenomic landscape in patient-derived induced pluripotent stem cells [Affymetrix]"
!Series_summary	"Hutchinson-Gilford Progeria Syndrome (HGPS) is a segmental premature aging disorder caused by the accumulation of the truncated form of Lamin A known as Progerin within the nuclear lamina.  Cellular hallmarks of HGPS include nuclear blebbing, loss of peripheral heterochromatin, defective epigenetic inheritance, altered gene expression, and senescence. To model HGPS using iPSCs, detailed genome-wide and structural analysis of the epigenetic landscape is required to assess the initiation and progression of the disease. We generated a library of iPSC lines from fibroblasts of HGPS patients and controls, including one family trio. Our microarray results suggest that despite the presence of defects associated with Progerin in the nuclear lamina including structural defects, senescenc

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Step 1: Determine if gene expression data is available
is_gene_available = True  # Based on the dataset description, it contains gene expression data

# Step 2: Determine data availability and convert data types

# Find the keys in the sample characteristics dictionary
trait_row = 2 if len(set(['condition: HGPS', 'condition: Normal'])) > 1 else None
gender_row = 0 if len(set(['Sex: Male', 'Sex: Female', 'Sex: ?'])) > 1 else None
age_row = None  # Age information is not directly available in the sample characteristics

# Define the conversion functions
def convert_trait(value):
    val = value.split(':')[-1].strip()
    if val == 'HGPS':
        return 1
    elif val == 'Normal':
        return 0
    else:
        return None

def convert_gender(value):
    val = value.split(':')[-1].strip()
    if val == 'Male':
        return 1
    elif val == 'Female':
        return 0
    else:
        return None

# Since age_row is None, we don't need to define a function for it
convert_age = None

# Step 3: Save cohort information
save_cohort_info('GSE84351', './preprocessed/Hutchinson-Gilford_Progeria_Syndrome/cohort_info.json', is_gene_available, trait_row is not None)

# Step 4: Clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Hutchinson-Gilford_Progeria_Syndrome', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Hutchinson-Gilford_Progeria_Syndrome/trait_data/GSE84351.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM2232606': [1, 1], 'GSM2232607': [1, 1], 'GSM2232608': [1, 1], 'GSM2232609': [1, 1], 'GSM2232610': [1, 1], 'GSM2232611': [1, 1], 'GSM2232612': [1, 1], 'GSM2232613': [1, 1], 'GSM2232614': [1, 1], 'GSM2232615': [0, 0], 'GSM2232616': [0, 0], 'GSM2232617': [0, 0], 'GSM2232618': [0, 0], 'GSM2232619': [0, 0], 'GSM2232620': [0, 0], 'GSM2232621': [0, 0], 'GSM2232622': [0, 0], 'GSM2232623': [0, 0], 'GSM2232624': [1, 1], 'GSM2232625': [1, 1], 'GSM2232626': [1, 1], 'GSM2232627': [1, 1], 'GSM2232628': [1, 1], 'GSM2232629': [1, 1], 'GSM2232630': [1, 1], 'GSM2232631': [1, 1], 'GSM2232632': [1, 1], 'GSM2232633': [0, 1], 'GSM2232634': [0, 1], 'GSM2232635': [0, 1], 'GSM2232636': [0, 1], 'GSM2232637': [0, 1], 'GSM2232638': [0, 1], 'GSM2232639': [0, 1], 'GSM2232640': [0, 1], 'GSM2232641': [1, 0], 'GSM2232642': [1, 0], 'GSM2232643': [1, 0], 'GSM2232644': [1, 0], 'GSM2232645': [1, 0], 'GSM2232646': [1, 0], 'GSM2232647': [1, 0], 'GSM2232648': [1, 0], 'GSM2232649': [0, 1], 'GSM2232650': [0, 1], 'GSM22326

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['16650001', '16650003', '16650005', '16650007', '16650009', '16650011',
       '16650013', '16650015', '16650017', '16650019', '16650021', '16650023',
       '16650025', '16650027', '16650029', '16650031', '16650033', '16650035',
       '16650037', '16650041'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': [12190.0, 29554.0, 69091.0, 160446.0, 317811.0], 'RANGE_END': [13639.0, 31109.0, 70008.0, 161525.0, 328581.0], 'total_probes': [25.0, 28.0, 8.0, 13.0, 36.0], 'GB_ACC': ['NR_046018', nan, nan, nan, 'NR_024368'], 'SPOT_ID': ['chr1:12190-13639', 'chr1:29554-31109', 'chr1:69091-70008', 'chr1:160446-161525', 'chr1:317811-328581'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Using the dictionary preview, determine the keys for mapping
identifier_key = 'ID'
gene_symbol_key = 'GB_ACC'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Preview the first few rows of the resulting gene_data to verify the mapping
print("Preview of gene expression data after mapping:")
print(gene_data.head())


Preview of gene expression data after mapping:
           GSM2232606  GSM2232607  GSM2232608  GSM2232609  GSM2232610  \
Gene                                                                    
NM_000014    5.446975    7.741524    5.742223    5.388367    7.050524   
NM_000015    2.339716    2.129192    2.698215    2.646941    2.162960   
NM_000019    5.364910    5.260948    6.018564    5.059661    5.379625   
NM_000020    5.768539    5.380835    5.656968    5.394048    5.347954   
NM_000021    6.730368    6.919831    6.848167    6.604000    6.849821   

           GSM2232611  GSM2232612  GSM2232613  GSM2232614  GSM2232615  ...  \
Gene                                                                   ...   
NM_000014    4.854656    5.749659    5.822712    5.633654    4.416880  ...   
NM_000015    2.199661    2.311718    2.509547    2.190749    2.231781  ...   
NM_000019    6.097930    6.614963    6.429518    6.238900    5.954911  ...   
NM_000020    5.828293    6.459803    6.358990    6.

### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Hutchinson-Gilford_Progeria_Syndrome/gene_data/GSE84351.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Hutchinson-Gilford_Progeria_Syndrome')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE84351', './preprocessed/Hutchinson-Gilford_Progeria_Syndrome/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Hutchinson-Gilford_Progeria_Syndrome/GSE84351.csv'
    unbiased_merged_data.to_csv(csv_path)


No gene data in the dataframe
